Copyright (c) Microsoft Corporation. All rights reserved.

Licensed under the MIT License.

# Transfer learning sample
* Train model
* Build Image 
* Create IoT Edge deployment JSON
* Deploy Model

![title](aml_config/AMLimages/VAIDKe2e.png)

In [ ]:
# For prod
!pip install azureml-core azureml-contrib-iot azure-mgmt-containerregistry azure-cli
!az extension add --name azure-cli-iot-ext

In [ ]:
import os
print(os.__file__)

In [ ]:
# Check core SDK version number
import azureml.core as azcore

print("SDK version:", azcore.VERSION)

### Create a Workspace
#### Change this cell from markdown to code and run this if you need to create a workspace 
#### Update the values for your workspace below
ws=Workspace.create(subscription_id="replace-with-subscription-id",
                resource_group="your-resource-group",
                name="your-workspace-name",
                location="eastus2")
                
ws.write_config()

In [ ]:
#Initialize Workspace 
from azureml.core import Workspace

ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

### Create Experiment
Experiment is a logical container in an Azure ML Workspace. It hosts run records which can include run metrics and output artifacts from your experiments.

In [ ]:
experiment_name = 'soda_cans'

from azureml.core import Experiment
exp = Experiment(workspace = ws, name = experiment_name)

## Get data
### Option 1: Upload data files into datastore
Every workspace comes with a default datastore (and you can register more) which is backed by the Azure blob storage account associated with the workspace. We can use it to transfer data from local to the cloud, and access it from the compute target.

In [ ]:
# get the default datastore
ds = ws.get_default_datastore()
print(ds.name, ds.datastore_type, ds.account_name, ds.container_name)

In [ ]:
data_path = experiment_name + '_training_data'
ds.upload(src_dir='data/soda_cans', target_path=data_path, overwrite=True)

### Option 2: Use existing datastore in Azure blob storage

In [ ]:
from azureml.core.datastore import Datastore
ds = Datastore.register_azure_blob_container(workspace=ws, 
                                         datastore_name='xxx', 
                                         container_name='xxx',
                                         account_name='xxxx', 
                                         account_key='xxx',
                                         create_if_not_exists=False)
data_path = "soda_cans_training_data" # This is the path to the folder in the blob container. Set this to None to get all the contents.
print(ds.name, ds.datastore_type, ds.account_name, ds.container_name)

### Configure for using ACI
Linux-based ACI is available in West US, East US, West Europe, North Europe, West US 2, Southeast Asia, Australia East, East US 2, and Central US regions.  See details [here](https://docs.microsoft.com/en-us/azure/container-instances/container-instances-quotas#region-availability)

In [ ]:
from azureml.core.runconfig import DataReferenceConfiguration
dr = DataReferenceConfiguration(datastore_name=ds.name, 
                   path_on_datastore=data_path, 
                   mode='download', # download files from datastore to compute target
                   overwrite=True)

Set the system to build a conda environment based on the run configuration. Once the environment is built, and if you don't change your dependencies, it will be reused in subsequent runs.

In [ ]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# choose a name for your cluster
cluster_name = "cpucluster1"

try:
    compute_target = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing compute target.')
except ComputeTargetException:
    print('Creating a new compute target...')
    compute_config = AmlCompute.provisioning_configuration(vm_size='Standard_D3', max_nodes=2)

    # create the cluster
    compute_target = ComputeTarget.create(ws, cluster_name, compute_config)

    compute_target.wait_for_completion(show_output=True)

# Use the 'status' property to get a detailed status for the current AmlCompute. 
print(compute_target.status.serialize())

In [ ]:
from azureml.core.runconfig import RunConfiguration, DEFAULT_CPU_IMAGE
from azureml.core.conda_dependencies import CondaDependencies

# create a new runconfig object
run_config = RunConfiguration(framework = "python")

# Set compute target
run_config.target = compute_target.name

# set the data reference of the run configuration
run_config.data_references = {ds.name: dr}

# enable Docker 
run_config.environment.docker.enabled = True

# set Docker base image to the default CPU-based image
run_config.environment.docker.base_image = DEFAULT_CPU_IMAGE

# use conda_dependencies.yml to create a conda environment in the Docker image for execution
run_config.environment.python.user_managed_dependencies = False

# auto-prepare the Docker image when used for execution (if it is not already prepared)
run_config.auto_prepare_environment = True

# specify CondaDependencies obj
run_config.environment.python.conda_dependencies = CondaDependencies.create(conda_packages=['tensorflow==1.8.0'])

### Submit the Experiment
Submit script to run in the Docker image in the remote VM. If you run this for the first time, the system will download the base image, layer in packages specified in the conda_dependencies.yml file on top of the base image, create a container and then execute the script in the container.

In [ ]:
from azureml.core import Run
from azureml.core import ScriptRunConfig

src = ScriptRunConfig(source_directory = './02-mobilenet-transfer-learning-scripts', script = 'retrain.py', run_config = run_config, 
                      # pass the datastore reference as a parameter to the training script
                      arguments=['--image_dir', str(ds.as_download()),
                                 '--architecture', 'mobilenet_1.0_224',
                                 '--output_graph', 'outputs/retrained_graph.pb',
                                 '--output_labels', 'outputs/output_labels.txt',
                                 '--model_download_url', 'https://raw.githubusercontent.com/rakelkar/models/master/model_output/',
                                 '--model_file_name', 'imagenet_2_frozen.pb'
                                ])
run = exp.submit(config=src)

In [ ]:
run

In [ ]:
run.wait_for_completion(show_output=True)

## Get the trained model

In [ ]:
trained_model_path = "models/mobilenet-orig/"

In [ ]:
# Download the retrained model and the labels locally
run.download_file(name = 'outputs/retrained_graph.pb', output_file_path = trained_model_path)
run.download_file(name = 'outputs/output_labels.txt', output_file_path = trained_model_path)

![title](aml_config/AMLimages/Image.png)

### VAM config file
This step uses the trained model from your local folder in the Notebooks shell.
There are three files (i) the __model_name.pb__ file, (ii) the __lables_names.txt__ and (iii) __va-snpe-engine-library_config.json__, in this folder. 

This va-snpe-engine-library_config file is used by the camera when loading the model into the inference engine.

__key fields are:__
1. Engine: This is the network used by the model
   * 0: MobileNet
   * 1: MobileNet-SSD 
   * 2: SqueezeNet
2. NetworkIO: 
   * 0: CPU (default)
   * 1: DSP
3. Runtime: this is the HW option to use for inferencing
   * 0: CPU 
   * 1: DSP 
   * 2: GPU
ConfThreshold: This is the threshold for when the bounding boxes are shown or inferencing results are shown on screen.

In [ ]:
vam_config_file = trained_model_path + "/va-snpe-engine-library_config.json"

In [ ]:
%%writefile $vam_config_file
{
"Engine":0,
"NetworkIO":1,
"ScaleWidth":224,
"ScaleHeight":224,
"PixelNorm":127.5,
"BlueMean":104,
"GreenMean":117,
"RedMean":123,
"TargetFPS":30,
"ConfThreshold":0.0,
"DLC_NAME":"model.dlc",
"LABELS_NAME":"output_labels.txt",
"InputLayers":"input:0",
"OutputLayers":["final_result"],
"ResultLayers":["final_result:0"],
"Runtime":1
}

In [ ]:
from azureml.core.model import Model

model = Model.register(model_path = trained_model_path,
                      model_name = "soda_cans",
                      tags = {"data": "Imagenet", "model": "object_detection", "type": "imagenet"},
                      description = "Retrained soda cans based on MobileNet",
                      workspace = ws)

In [ ]:
from azureml.contrib.iot.model_converters import SnpeConverter

# submit a compile request
compile_request = SnpeConverter.convert_tf_model(
    ws,
    source_model=model,
    input_node="input",
    input_dims="1,224,224,3",
    outputs_nodes = ["final_result"],
    allow_unconsumed_nodes = True)
print(compile_request._operation_id)

In [ ]:
# wait for the request to complete
compile_request.wait_for_completion(show_output=True)

In [ ]:
# Get converted model
converted_model = compile_request.result
print(converted_model.name, converted_model.url, converted_model.version, converted_model.id, converted_model.created_time)

In [ ]:
from azureml.core.image import Image
from azureml.contrib.iot import IotContainerImage
print ('We will create an image for you now ...')
image_config = IotContainerImage.image_configuration(
                                 architecture="arm32v7",
                                 execution_script="main.py", 
                                 dependencies=["camera.py","iot.py","ipcprovider.py","utility.py", "frame_iterators.py"],
                                 docker_file="Dockerfile", 
                                 tags = ["mobilenet"],
                                 description = "MobileNet model retrained soda cans")
image = Image.create(name = "mobilenetsoda",
                     # this is the model object 
                     models = [converted_model],
                     image_config = image_config, 
                     workspace = ws)
image.wait_for_creation(show_output = True)

![title](aml_config/AMLimages/Deploy.png)

#### Setup the details for the IoT Hub. 
You can use the configuration from an existing IoT Hub if you have one setup.

In [ ]:
# Parameter list

# Pick a name for what you want to call the module you deploy to the camera
module_name = "visionsample"

# Resource group in Azure 
resource_group_name= ws.resource_group
iot_rg="vaidk_"+resource_group_name # or use the existing RG

# Azure region where your services will be provisioned
iot_location="eastus2" # or use the existing location

# Azure IoT Hub name
iot_hub_name="iothub-"+ ws.get_details()["name"] # or use the name of an existing IoT Hub

# Pick a name for your camera
iot_device_id="vadik_"+ ws.get_details()["name"] # or use the existing device ID from IoT Hub configuration

# Pick a name for the deployment configuration
iot_deployment_id="demovaidk" # 

In [ ]:
# Getting your container details
container_reg = ws.get_details()["containerRegistry"]
reg_name=container_reg.split("/")[-1]
container_url = "\"" + image.image_location + "\","
subscription_id = ws.subscription_id
print('{}'.format(image.image_location))
print('{}'.format(reg_name))
print('{}'.format(subscription_id))
from azure.mgmt.containerregistry import ContainerRegistryManagementClient
from azure.mgmt import containerregistry
client = ContainerRegistryManagementClient(ws._auth,subscription_id)
result= client.registries.list_credentials(resource_group_name, reg_name, custom_headers=None, raw=False)
username = result.username
password = result.passwords[0].value

### Deployment file
This is the deployment.json file that you will use to deploy your model.  Please see the other sample notebooks on using this file to deploy the new model you created.

In [ ]:
file = open('./deployment-template.json')
contents = file.read()
contents = contents.replace('__MODULE_NAME', module_name)
contents = contents.replace('__REGISTRY_NAME', reg_name)
contents = contents.replace('__REGISTRY_USER_NAME', username)
contents = contents.replace('__REGISTRY_PASSWORD', password)
contents = contents.replace('__REGISTRY_IMAGE_LOCATION', image.image_location)
with open('./deployment.json', 'wt', encoding='utf-8') as output_file:
    output_file.write(contents)

### Option 1: Push this updated deployment manifest to an existing IoT Hub

In [ ]:
!az login

In [ ]:
# set azure subscription
with open ('setsub','w+') as command1:
    command1.write('az account set --subscription ' + subscription_id )
    command1.close()
!sh setsub

In [ ]:
# confirm the default azure account before pushing IoT Edge deployment
!az account show

In [ ]:
print("Pushing deployment to IoT Edge device")

In [ ]:
print ("Set the deployment") 
!az iot edge set-modules --device-id $iot_device_id --hub-name $iot_hub_name --content deployment.json

### Option 2: Create a new Azure IoT Hub
__SKIP__ if you have already pushed to an existing IoT Hub

In [ ]:
# print ( 'We will create your HUB now')

# with open ('setsub','w+') as command1:
#     command1.write('az account set --subscription ' + subscription_id )
#     command1.close()
# !sh setsub

# with open ('create','w+') as command2:
#     regcommand="\n echo Installing Extension ... \naz extension add --name azure-cli-iot-ext \n"+ "\n echo CREATING RG "+iot_rg+"... \naz group create --name "+ iot_rg +" --location "+ iot_location+ "\n" +"\n echo CREATING HUB "+iot_hub_name+"... \naz iot hub create --name "+ iot_hub_name + " --resource-group "+ iot_rg +" --sku S1"
#     command2.write(regcommand +"\n echo CREATING DEVICE ID "+iot_device_id+"... \n az iot hub device-identity create --device-id "+ iot_device_id + " --hub-name " +  iot_hub_name +" --edge-enabled")
#     command2.close()
# !sh create

# with open ('deploy','w+')as command3:
#     createcommand="\n echo DEPLOYING "+iot_deployment_id+" ... \naz iot edge deployment create --deployment-id \"" + iot_deployment_id + "\" --content \"deployment.json\" --hub-name \"" +  iot_hub_name +"\" --target-condition \"deviceId='"+iot_device_id+"'\" --priority 1"
#     command3.write(createcommand)
#     command3.close()
# !sh deploy

# with open ('details','w+')as command4:
#     get_string="\n echo THIS IS YOUR CONNECTION STRING ... \naz iot hub device-identity show-connection-string --device-id  \"" + iot_device_id + "\" --hub-name \"" +  iot_hub_name+"\""
#     command4.write(get_string)
#     command4.close()

In [ ]:
# !sh details
# print("COPY THIS CONNECTION STRING FOR YOUR DEVICE")

#### Now use this conenction string on your Vision AI Dev Kit during the device OOBE & initialization